## 1. Tokenization

### Load text file

In [1]:
with open("the-verdict.txt", "r", encoding = "utf-8") as t:
    raw_text = t.read()

print(f"Total number of characters: {len(raw_text)}")
print(raw_text[:99])

Total number of characters: 20479
I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no 


### RE tokenizer

In [2]:
import re

sample = "Hey, what's good?"
result = re.split(r'([,.:;?!"()\'/]|--|\s)', sample)

print(result)

['Hey', ',', '', ' ', 'what', "'", 's', ' ', 'good', '?', '']


In [3]:
result = [item for item in result if item.strip()] # retunrs false for whitespaces / no spaces
print(result)

['Hey', ',', 'what', "'", 's', 'good', '?']


Now, apply RE tokenizer to main text.

In [4]:
preprocessed = re.split(r'([.,:;?!"()\'/]|--|\s)', raw_text)
preprocessed = [item for item in preprocessed if item.split()]

print(f"Number of tokens: {len(preprocessed)}")
print(preprocessed[:30])

Number of tokens: 4654
['I', 'HAD', 'always', 'thought', 'Jack', 'Gisburn', 'rather', 'a', 'cheap', 'genius', '--', 'though', 'a', 'good', 'fellow', 'enough', '--', 'so', 'it', 'was', 'no', 'great', 'surprise', 'to', 'me', 'to', 'hear', 'that', ',', 'in']


### Token ID creation

In [5]:
all_words = sorted(set(preprocessed))
print(f"Length of vocabulary: {len(all_words)}")

Length of vocabulary: 1139


In [6]:
vocab = {token:integer for integer, token in enumerate(all_words)}

for t, i in vocab.items():
    print(t, i)
    if i >= 20:
        break

! 0
" 1
' 2
( 3
) 4
, 5
-- 6
. 7
: 8
; 9
? 10
A 11
Ah 12
Among 13
And 14
Are 15
Arrt 16
As 17
At 18
Be 19
Begin 20


### Tokenizer class

In [7]:
class TokenizerV1:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = {i:s for s, i in vocab.items()}

    def encode(self, text):
        preprocessed = re.split(r'([,.:;?_!"()\'/]|--|\s)', text)
        preprocessed = [
            item.strip() for item in preprocessed if item.strip() # remove white spaces
        ]
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids
    
    def decode(self, ids):
        text = " ".join(self.int_to_str[i] for i in ids) # int_to_str gives back the text in a list. " ".join joins them into a normal sentence
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text) # fixes space before punctuations
        return text

In [8]:
tokenizer = TokenizerV1(vocab)
s2i = tokenizer.encode("I HAD always thought.")
tokenizer.decode(s2i)

'I HAD always thought.'

### Special Context Tokens

In [9]:
all_tokens = sorted(list(set(preprocessed)))
all_tokens.extend(['<|endoftext|>', '<|unk|>'])

vocab = {token:integer for integer, token in enumerate(all_tokens)}
print(f"Length of vocabulary with special context tokens: {len(vocab.items())}")

Length of vocabulary with special context tokens: 1141


In [10]:
keys = []
for k, v in enumerate(vocab.keys()):
    keys.append(v)
print(keys[-5:])

['younger', 'your', 'yourself', '<|endoftext|>', '<|unk|>']


In [11]:
class TokenizerV2:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = {i:s for s, i in vocab.items()}

    def encode(self, text):
        preprocessed = re.split(r'([,.:;?_!"()\'/]|--|\s)', text)
        preprocessed = [
            item.strip() for item in preprocessed if item.strip() # remove white spaces
        ]
        # if item not in vocab, replace it with <|unk|> token
        preprocessed = [
            item if item in self.str_to_int
            else "<|unk|>" for item in preprocessed
        ]
        preprocessed.append("<|endoftext|>")
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids
    
    def decode(self, ids):
        text = " ".join(self.int_to_str[i] for i in ids) # int_to_str gives back the text in a list. " ".join joins them into a normal sentence
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text) # fixes space before punctuations
        return text

In [12]:
tokenizer = TokenizerV2(vocab)
s2i = tokenizer.encode("Hello, how are you doing?")
tokenizer.decode(s2i)

'<|unk|>, how are you doing? <|endoftext|>'

### Byte Pair Encoding

In [13]:
import importlib
import tiktoken

In [14]:
tokenizer = tiktoken.get_encoding('gpt2')

In [15]:
text = "Hello, Ilham. <|endoftext|> Would you like a cupoftea?"
integers = tokenizer.encode(text, allowed_special = {'<|endoftext|>'})
print(integers)

[15496, 11, 13778, 2763, 13, 220, 50256, 10928, 345, 588, 257, 6508, 1659, 660, 64, 30]


In [16]:
strings = tokenizer.decode(integers)
print(strings)

Hello, Ilham. <|endoftext|> Would you like a cupoftea?


## 2. Input-Target Pairs

In [17]:
with open("the-verdict.txt", "r", encoding = "utf-8") as f:
    raw_text = f.read()

enc_text = tokenizer.encode(raw_text)
print("Total number of tokens from byte pair encoding:", len(enc_text))

Total number of tokens from byte pair encoding: 5145


In [18]:
context_size = 5 # input will have 5 tokens
x = enc_text[:context_size]
y = enc_text[1:context_size + 1]
print(f"X: {x}")
print(f"y:     {y}")

X: [40, 367, 2885, 1464, 1807]
y:     [367, 2885, 1464, 1807, 3619]


### Using Dataloader

In [19]:
from torch.utils.data import Dataset, DataLoader

class GPTDatasetV1(Dataset):
    def __init__(self, txt, tokenizer, max_len, stride): # max_len is context size
        self.input_ids = []
        self.target_ids = []

        # tokenize the text
        token_ids = tokenizer.encode(txt, allowed_special = {"<|endoftext|>"})

        # sliding window to create overlapping sequences
        for i in range(0, len(token_ids) - max_len, stride):
            input_chunk = token_ids[i:i + max_len]
            target_chunk = token_ids[i + 1:i + max_len + 1]
            self.input_ids.append(input_chunk)
            self.target_ids.append(target_chunk)
    
    # the below 2 methods is required for Dataloader to be used
    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx): # we are basically saying that if the input is the 50th tensor, then the output is the 50th tensor
        return self.input_ids[idx], self.target_ids[idx]

The idea is to form something like as follows:<br><br>
[[1, 2, 3, 4],<br>
[5, 6, 7, 8],<br>
[9, 10, 11, 12]]<br><br>
[[2, 3, 4, 5],<br>
[6, 7, 8, 9],<br>
[10, 11, 12, 13]]<br><br>
...where the first matrix is X and the second matrix is y. Note that in the above example, the stride as well as the max length is 4. If the stride was 2:<br><br>
[[1, 2, 3, 4],<br>
[3, 4, 5, 6],<br>
[5, 6, 7, 8]]<br><br>
[[2, 3, 4, 5],<br>
[4, 5, 6, 7],<br>
[6, 7, 8, 9]]

In [20]:
def create_dataloader_v1(txt, batch_size = 4, max_len = 256, stride = 128, shuffle = True, drop_last = True, num_workers = 0):
    # drop last if last tensor is shorter than max_len
    # batch size is the number of training ip-op data pairs to be used for training by whcih the parameters are updated
    tokenizer = tiktoken.get_encoding('gpt2')
    dataset = GPTDatasetV1(txt, tokenizer, max_len, stride)
    dataloader = DataLoader(
        dataset, 
        batch_size = batch_size,
        shuffle = shuffle,
        drop_last = drop_last,
        num_workers = num_workers
    )
    return dataloader

In [21]:
with open("the-verdict.txt", "r", encoding = "utf-8") as f:
    raw_text = f.read()

In [22]:
import torch
dataloader = create_dataloader_v1(raw_text, batch_size = 1, max_len = 4, stride = 1, shuffle = False) # looking into how the function will work
data_iter = iter(dataloader)
first_batch = next(data_iter)
print(first_batch)

[[tensor([40]), tensor([367]), tensor([2885]), tensor([1464])], [tensor([367]), tensor([2885]), tensor([1464]), tensor([1807])]]


Using a batch size of 1 is not preferred as this leads to noisy updates, even though good for memory.<br>
Note that a higher overlap (lower stride) can lead to overfitting.

## 3. Vector Embeddings

In [25]:
# sample

input_ids = torch.tensor([2, 3, 5, 1])
vocab_szie = 6
output_dim = 3 # embedding dimention

torch.manual_seed(123)
embedding_layer = torch.nn.Embedding(vocab_szie, output_dim) # intialize mebedding matrix randomly
print(embedding_layer.weight)

Parameter containing:
tensor([[ 0.3374, -0.1778, -0.1690],
        [ 0.9178,  1.5810,  1.3010],
        [ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-1.1589,  0.3255, -0.6315],
        [-2.8400, -0.7849, -1.4096]], requires_grad=True)


In [29]:
print(embedding_layer(torch.tensor([3])))

tensor([[-0.4015,  0.9666, -1.1481]], grad_fn=<EmbeddingBackward0>)


The embedding weight matrix is basically used for lookup operation.

In [34]:
print(embedding_layer(input_ids)) # looking up vector embeddings for the sample input

tensor([[ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-2.8400, -0.7849, -1.4096],
        [ 0.9178,  1.5810,  1.3010]], grad_fn=<EmbeddingBackward0>)


Note that this is essencially a one hot encoded represenation of the input IDs passed into a linear layer to get the output embeddings where the weights of the neural net are randomly initialized. But we dom't use this because it's not efficient due to the sparsity of the one hot encoded input matrix.